In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# all columns
# name scheme: columns-num_cycles-sl-noise
from config import *
context = Context('2-23-redundent-features', fn_cycles='tbmData/cycles1', debug=1, mulr=0, load_data=False)

In [ ]:
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles # shorten notation

# idx = fmtr.beginning_index()
# df_conts, stat_x = fmtr.get_x()
# deps, stat_y = fmtr.get_y(normalize=True)
# train_cont = flatten_and_cat(df_conts, deps, sl=context.sl)

# Drop non-numeric columns

Only '运行时间' is non-numeric. 

But there seems to be nan values in the the column.

In [ ]:
cycles.loc[:,['运行时间']].iloc[1280:1300].plot()

In [ ]:
cycles.loc[:,['']].iloc[1280:1300].plot()

In [ ]:
cycles.iloc[1280:1300]

I don't see any problems with that. Probably because of type conversion from string.

In [ ]:
cycles.drop(columns=kkkkjjkjkjk, inplace=True)

# Drop inf 

No inf and -inf

In [ ]:
(cycles == -np.inf).sum().sort_values(ascending=False) + (cycles == np.inf).sum().sort_values(ascending=False)

# Drop constant columns

In [ ]:
std = cycles.std()

In [ ]:
std.sort_values()

In [ ]:
cycles.drop(columns=cycles.columns[std == 0], inplace=True)

In [ ]:
std = cycles.std().sort_values(); std

Look at columns with low variance

In [ ]:
plots(cycles.loc[:,std.index[:3]]);

In [ ]:
plots(cycles.loc[:,std.index[:3]].iloc[:1000]);

Looks fine

## Before dropping numbered columns, average them

In [ ]:
prog = re.compile('.*\d*#.*')
average_columns = [s for s in cycles.columns if re.match(prog, s) is not None]; average_columns
for i in range(0, len(average_columns), 10):
    # group columns of similar sensors
    g = average_columns[i:i+10]
    new_column = re.sub('\d#', '', g[0])
    cycles.loc[:,g[0]] = cycles.loc[:,g].mean(1)
    cycles.drop(columns=g[1:], inplace=True)
    cycles.rename(columns={g[0]: new_column}, inplace=True, copy=False)

# Drop columns with high correlation

Protect '桩号' because 用来训练地质预测模型. Remember to exclude this column from input data.

In [ ]:
dep_var2 = ['总推进力', '刀盘扭矩']
protected_columns = ['桩号'] + dep_var1 + dep_var2
protected_columns

Think about how to protect certain columns from being dropped. If columns i and j (i < j) are highly correlated, we only drop one of them by masking the correlation matrix with it's upper triangle, where the first index is smaller.

Thus we only see correlation [i, j] in the masked matrix. And because we drop columns according the the high correlation value's column number, column i is protected from being dropped.

In conclusion, to protect certain columns from being dropped, we should move it forward before computing the correlation matrix to assign it a smaller index number (which is i in the analysis).

In [ ]:
reordered_columns = protected_columns + [c for c in cycles.columns if c not in protected_columns]
cycles = cycles.loc[:, reordered_columns]

In [ ]:
# https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
# Create correlation matrix
corr_matrix = cycles.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [c for c in upper.columns if np.any(upper.loc[:,c] > 0.95)]

In [ ]:
to_drop

In [ ]:
[o in to_drop for o in protected_columns]

In [ ]:
display_all(upper.loc[:,to_drop])

In [ ]:
cycles.drop(columns=to_drop, inplace=True)

In [ ]:
# https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
# Create correlation matrix
corr_matrix = cycles.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [ ]:
display_all(upper.max())